In [58]:
# knn = neighbors.KNeighborsRegressor(n_neighbors = 5, weights= 'uniform')
# y_pred = knn.fit(insample_data.drop(, 1), insample_data[[stock]]).predict(outsample_data.drop([stock], 1))

In [1]:
# data = (clean_returns - clean_returns.mean(axis=0))/clean_returns.std(axis=0)
# model = decomposition.FactorAnalysis(n_components=2,tol=1e-8,max_iter=1000)
# model.fit(data)
# Y = model.fit_transform(data)
# data_comp = pd.DataFrame(np.dot(Y, (model.components_)), columns = data.columns, index = data.index)
# fa_weights = pd.DataFrame(model.components_, columns = data.columns)

In [35]:
import pandas as pd
import numpy as np
import random
import os
from scipy import signal
import statsmodels
from scipy.stats import norm
from math import factorial
import time
from scipy import stats
import scipy as scipy
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.6f}'.format
np.set_printoptions(threshold=5)
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import figure
from sklearn import neighbors


In [3]:
import os
from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
from sklearn import decomposition
from sklearn.decomposition import SparsePCA

yf.pdr_override()

START_DATE = "2010-01-01"
END_DATE = "2015-01-01"

In [33]:
tickers_mapping = pd.read_csv('Tickers.csv',  encoding='latin-1')
tickers_sorted = tickers_mapping.sort_values(['Sector']).Symbol
tickers_mapping.index = tickers_mapping['Symbol']

prices = pd.read_csv('stocks.csv')
prices.index = pd.DatetimeIndex(prices['Unnamed: 0'])
prices = prices.iloc[:,1:]
index = pd.read_csv('index.csv')
index.index = pd.DatetimeIndex(index['Date'])
raw_prices = pd.concat([prices, index['Adj Close'].rename('INDEX')], axis=1)        

returns = raw_prices.pct_change(1)
std_returns = returns.div(returns.std(0), 1)
betas = ((returns.rolling(60).corr(returns['INDEX']))*\
                            (returns.rolling(60).std().div(returns['INDEX'].rolling(60).std(),axis = 0))).shift(1)
idio_returns = returns - betas.mul(returns['INDEX'], axis = 0)
idio_std_returns = idio_returns.div(idio_returns.std(0), 1)

clean_returns = returns.loc[:,tickers_sorted].dropna(0, how = 'all').dropna(1)
clean_idio_returns = idio_returns.loc[:,tickers_sorted].dropna(0, how = 'all').dropna(1)
clean_idio_std_returns = idio_std_returns.loc[:,tickers_sorted].dropna(0, how = 'all').dropna(1)

In [64]:
tmp_returns = std_returns.dropna(0, how = 'all').dropna(1, how = 'all')

In [158]:
def KNN(stock, n_neighbors):

    train_data = tmp_returns[tmp_returns[stock].isna() == False]
    insample_data = train_data.loc[random.sample(list(train_data.index), 3*len(train_data.index)//4)]
    outsample_data = train_data.loc[[i for i in train_data.index if i not in insample_data.index]]
    final_test_data = pd.concat([insample_data, outsample_data], 0)

    outsample_data[[stock]] = np.nan
    final_train_data = pd.concat([insample_data, outsample_data], 0)

    imputer = KNNImputer(n_neighbors = n_neighbors)
    fill_data = pd.DataFrame(imputer.fit_transform(final_train_data), \
                             columns = final_train_data.columns, index = final_train_data.index)
    print(stock, 'Error:', abs((fill_data[stock] - final_test_data[stock])[outsample_data.index]).median(), 
          'Mean:', abs((final_test_data[stock])[outsample_data.index]).median())
    
    return abs((fill_data[stock] - final_test_data[stock])[outsample_data.index]).median()


In [274]:
def KNN(stock, n_neighbors):

    tmp_returns_wo_stock = tmp_returns.drop([stock], 1).dropna(0, how = 'all').dropna(1)
    tmp_returns_w_stock = pd.concat([tmp_returns_wo_stock, \
                                  tmp_returns.loc[tmp_returns_wo_stock.index, stock]], 1)
    
    if tmp_returns_w_stock.isna().sum().sum() == 0:
        return 0 
    
    train_data = tmp_returns_w_stock[tmp_returns_w_stock[stock].isna() == False]
    insample_data = train_data.loc[random.sample(list(train_data.index), 3*len(train_data.index)//4)]
    outsample_data = train_data.loc[[i for i in train_data.index if i not in insample_data.index]]
    final_test_data = pd.concat([insample_data, outsample_data], 0)

    outsample_data[[stock]] = np.nan
    final_train_data = pd.concat([insample_data, outsample_data], 0)
    
    knn = neighbors.KNeighborsRegressor(n_neighbors = n_neighbors, metric='manhattan')
    fill_data = knn.fit(insample_data.drop([stock], 1), insample_data[[stock]]).predict(outsample_data.drop([stock], 1))
    fill_data = pd.DataFrame(fill_data, index=outsample_data.index, columns=[stock])
    
    print(stock, 'Error:', (abs((fill_data[stock] - final_test_data.loc[outsample_data.index, stock]))**1).median(), 
          'Mean:', abs((final_test_data.loc[outsample_data.index, stock])).median())
          
    imputer = KNNImputer(n_neighbors = n_neighbors)
    fill_data2 = pd.DataFrame(imputer.fit_transform(final_train_data), \
                             columns = final_train_data.columns, index = final_train_data.index)
    print(stock, 'Error:', (abs((fill_data2[stock] - final_test_data.loc[outsample_data.index, stock]))**1).median(), 
          'Mean:', abs((final_test_data.loc[outsample_data.index, stock])).median())

In [275]:
n_neighbors = 5
KNN('ABBV', n_neighbors)

ABBV Error: 0.5533165569159693 Mean: 0.6073743231892423
ABBV Error: 0.5108557978784567 Mean: 0.6073743231892423


In [276]:
stock_list = ['XYL', 'ZTS', 'ABBV']
neighbors_list = 10
table = pd.DataFrame(columns=stock_list)
for stock in stock_list:
    for n in range(1, neighbors_list):
        table.loc[n, [stock]] = KNN(stock, n)

XYL Error: 0.44808797342857 Mean: 0.5558836370802263
XYL Error: 0.48456455354595984 Mean: 0.5558836370802263
XYL Error: 0.4561111276484619 Mean: 0.5887028615455252
XYL Error: 0.4997986450453075 Mean: 0.5887028615455252
XYL Error: 0.4567592891525101 Mean: 0.6003840743963802
XYL Error: 0.49576809847954395 Mean: 0.6003840743963802
XYL Error: 0.4197927522580927 Mean: 0.5520586939403458
XYL Error: 0.41264747019736087 Mean: 0.5520586939403458
XYL Error: 0.4075839897382476 Mean: 0.5922653328669987
XYL Error: 0.43283008262161915 Mean: 0.5922653328669987
XYL Error: 0.40578952890057796 Mean: 0.5242788327323861
XYL Error: 0.3921504458179867 Mean: 0.5242788327323861
XYL Error: 0.5291712267530757 Mean: 0.6535219539219463
XYL Error: 0.5332341776162404 Mean: 0.6535219539219463
XYL Error: 0.41019467131935416 Mean: 0.5721850607151274
XYL Error: 0.4097728422445871 Mean: 0.5721850607151274
XYL Error: 0.4546297322701634 Mean: 0.6689045200515951
XYL Error: 0.4575775355302002 Mean: 0.6689045200515951
ZTS Er

In [262]:
table

,FDX,MCO
1,0,0
2,0,0
